In [1]:
import sys
sys.path.append('..')
sys.path.append('../src/fairseq')
sys.path.append('../src/fairseq/examples')
from src.unimodal import NLPDataset, AudioDataset
from fairseq.tasks.audio_pretraining import AudioMaskingConfig
from dataclasses import asdict

/Users/timcares/miniforge3/envs/mmrl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = '../data/language/enwik9'

In [3]:
ds = NLPDataset(data_path=data_path, num_max_bpe_tokens=512)

loaded 2 blocks from: ../data/language/enwik9/train


In [4]:
loader = ds.get_dataloader(batch_size=8, shuffle=True, num_workers=1)

In [5]:
loader_iter = iter(loader)

In [6]:
next(loader_iter)['net_input']

OrderedDict([('source',
              tensor([[    0, 32098,  3443,  ...,     1,     1,     1],
                      [    0,   260, 13161,  ...,   548,  8624,     2]])),
             ('padding_mask',
              tensor([[False, False, False,  ...,  True,  True,  True],
                      [False, False, False,  ..., False, False, False]]))])

In [2]:
audio_path = '../data/LibriSpeech'

In [11]:
am_cfg =  AudioMaskingConfig(
    feature_encoder_spec='[(512, 10, 5)] + [(512, 3, 2)] * 4 + [(512,2,2)] + [(512,2,2)]',
    mask_prob=0.5,
    mask_prob_adjust=0.05,
    mask_length=5,
    inverse_mask=False,
    mask_dropout=0,
    clone_batch=2,
)

In [12]:
audio_dataset = AudioDataset(data_path=audio_path, sample_rate=16000, max_sample_size=320000, min_sample_size=32000, precompute_mask_config=asdict(am_cfg))

In [13]:
audio_loader = audio_dataset.get_dataloader(batch_size=8, shuffle=True, num_workers=1)

In [14]:
audio_loader_iter = iter(audio_loader)

In [28]:
next(audio_loader_iter)['net_input']['source'].shape

torch.Size([8, 84080])

In [29]:
next(audio_loader_iter)

{'id': tensor([15551,  3939,  3716, 25035, 11659, 27998, 22352,  2101]),
 'net_input': {'source': tensor([[ 9.0226e-01,  8.9310e-01,  8.4690e-01,  ...,  2.1881e+00,
            2.1682e+00,  2.3574e+00],
          [ 4.9034e-01, -2.5139e-01, -2.2811e-02,  ..., -4.8209e-02,
           -4.6906e-02, -1.6738e-01],
          [-4.1562e-01, -5.9688e-01, -6.0068e-01,  ...,  1.2183e-01,
            1.6974e+00,  2.6506e+00],
          ...,
          [-3.7082e-03, -3.1989e-03, -2.6895e-03,  ...,  3.9766e-01,
            4.1854e-01,  4.0530e-01],
          [-1.2794e-01, -9.9036e-02, -5.5682e-02,  ..., -1.3812e+00,
           -1.8875e+00, -2.6442e+00],
          [-9.4794e-04, -9.4794e-04, -9.4794e-04,  ..., -9.4794e-04,
           -9.4794e-04, -9.4794e-04]]),
  'precomputed_mask': tensor([[0., 1., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 1., 0.],
